In [1]:
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from keras.preprocessing.sequence import pad_sequences
import nltk
import pandas as pd
import numpy as np
import os
import sys
import pprint
import itertools
import re
from tqdm.notebook import trange, tqdm
from spellchecker import SpellChecker
import html
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec
from transformers import BertModel, BertTokenizer
from keras.layers import Input,Dense,Conv2D,MaxPooling2D,UpSampling2D
from keras.layers import Flatten,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D
from keras.models import Model,Sequential
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
path_to_dataset = 'nlp-getting-started/'
test_df = pd.read_csv(os.path.join(path_to_dataset, 'test.csv'))
train_df = pd.read_csv(os.path.join(path_to_dataset, 'train.csv'))

stop_words = stopwords.words('english')

In [3]:
bertTknzr = BertTokenizer.from_pretrained('bert-base-uncased')
bert_wordlist = list(bertTknzr.vocab.keys())

In [4]:
word2vec_model = KeyedVectors.load_word2vec_format('word2vec_twitter_tokens.bin', binary=True, unicode_errors='ignore') 
#model = gensim.models.KeyedVectors.load_word2vec_format('word2vec_twitter_tokens.bin',binary=True，unicode_errors='ignore') 

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.options.mode.chained_assignment = None

Spell Check functions

In [6]:
def reduce_length(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def correct_spelling(text):
    spell = SpellChecker()
    reduced_text  = list([reduce_length(word) for word in text])
    corrected_text = [spell.correction(word) for word in reduced_text]
    print(reduced_text)
    misspelled = spell.unknown(reduced_text)
    print(misspelled)
    for word in misspelled:
        # Get the one `most likely` answer
        print(spell.correction(word))
        # Get a list of `likely` options
        print(spell.candidates(word))
    return corrected_text

Clean text funtions

In [7]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_punc(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def html_unescape(text):
    return html.unescape(text)

#tokenize sentence and find emoji stands for happy and sad
def token(text):
    tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
    words = [word for word in tknzr.tokenize(text)]
    for i in range(len(words)):
        if words[i] in word2vec_model:
            if word2vec_model.similarity(words[i],":)")>0.55:
                words[i] = 'happy'
            elif word2vec_model.similarity(words[i],":(")>0.55:
                words[i] = 'sad'
    #return words
    return ' '.join(words)

def reduce_length(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def correct_spelling(text):
    spell = SpellChecker()
    text = text.split()
    reduced_text  = [reduce_length(word) for word in text]
    corrected_text = [word if (word in bert_wordlist) else spell.correction(word) for word in reduced_text]
    return ' '.join(corrected_text)

def remove_digits(text): 
    pattern = '[0-9]'
    text = re.sub(pattern, '', text)
    return text

def textClean(df):
    df['text'] = df['text'].apply(lambda text: html.unescape(text))
    df['text'] = df['text'].str.lower()
    df['text'] = df['text'].apply(lambda text: remove_URL(text))
    df['text'] = df['text'].apply(lambda text: remove_digits(text))
    df['text'] = df['text'].apply(lambda text: remove_html(text))
    df['text'] = df['text'].apply(lambda text: token(text))
    df['text'] = df['text'].apply(lambda text: remove_punc(text))
    for i in tqdm(range(len(df['text']))):
        df['text'][i] = correct_spelling(df['text'][i])
    #df['text'] = df['text'].apply(lambda text: correct_spelling(text))
    
    return df

In [8]:
clean_train = textClean(train_df)

In [9]:
train_df.to_csv('clean_train.csv')

In [10]:
clean_test = textClean(test_df)

In [11]:
test_df.to_csv('clean_test.csv')